In [1]:
import os
import ast
import openai
import time
import json
from json import JSONDecodeError
import re

In [2]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = "" #My Key
# openai.Model.list()

# # engines = openai.Engine.list()
# # print(engines)

<OpenAIObject list at 0x2675e5546d0> JSON: {
  "object": "list",
  "data": [
    {
      "id": "dall-e-3",
      "object": "model",
      "created": 1698785189,
      "owned_by": "system"
    },
    {
      "id": "dall-e-2",
      "object": "model",
      "created": 1698798177,
      "owned_by": "system"
    },
    {
      "id": "gpt-3.5-turbo-0125",
      "object": "model",
      "created": 1706048358,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-ada-002",
      "object": "model",
      "created": 1671217299,
      "owned_by": "openai-internal"
    },
    {
      "id": "tts-1-hd-1106",
      "object": "model",
      "created": 1699053533,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-3-small",
      "object": "model",
      "created": 1705948997,
      "owned_by": "system"
    },
    {
      "id": "gpt-4-0613",
      "object": "model",
      "created": 1686588896,
      "owned_by": "openai"
    },
    {
      "id": "tts-1-hd",
      "object

In [3]:
with open('llm_summary_by_gpt35.json', 'r') as f:
    data = json.load(f)
print(len(data))

1200


In [4]:
with open('evidence_corpus.json', 'r', encoding='utf-8') as f:
    evidence_data = json.load(f)
print(len(evidence_data))

8096


In [5]:
# Call GPT3.5 Model 
# def get_completion_from_messages(messages, 
#                                  model="gpt-4-1106-preview", 
#                                  temperature=0, 
#                                  max_tokens=1024):
#     try:
#         response = openai.ChatCompletion.create(
#             model=model,
#             messages=messages,
#             temperature=temperature, # this is the degree of randomness of the model's output
#             max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
#         )
#         return response.choices[0].message["content"]
#     except openai.error.RateLimitError:
#         return None

# ChatCompletion.create
# from openai import OpenAI
# client = OpenAI()
# openai.chat.completions.create


# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
                                 model= "gpt-3.5-turbo-16k-0613",
                                 temperature=0, 
                                 max_tokens=1024):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        format={"type": "json_object"}
    )
    return response.choices[0].message["content"]

In [6]:
delimiter_1 = "#"
delimiter_2 = "##"

# 3. the factual accuracy of the news story based on these evidence sentences. Are they supporting or contradicting the news?

system_message = f"""
As a Fact Checker, your role involves analyzing a news paragraph and several evidence abstracts provided by the user.\
Here is how you should approach this task:
1. The user will present a news paragraph. This will be marked with the delimiter {delimiter_1}. Carefully read this paragraph to understand its central claim.
2. Following this, the user will present evidence abstracts, marked with the delimiter {delimiter_2}. These abstracts may either support or refute the news paragraph's central claim.
3. Your task is to determine whether the news story is factually true or false based on the evidence abstracts.\ 
4. To justify your conclusion, select specific sentences from both the news story and the evidence provided.
Remember, your analysis should be thorough and objective, focusing on the factual alignment between the news paragraph and the evidence sentences.
"""

# system_message = f"""
# You are a Fact Checker.\ The user will give you a news paragraph and evidence.\
# The evidence may contain information that upholds or contradicts the news article’s central claim. \
# Determine whether the news story is factually true or fake based on the evidence.\ 
# The user will put the news paragraph behind {delimiter_1} and the relevant abstracts behind {delimiter_2}. 
# """

In [7]:
# news_paragraph_1 = f"""
# A new study published in the journal PLOS Medicine has found that geolocation strategies can accurately track disease distribution in rural areas where formal address networks do not exist. The study, conducted in western Kenya, tested three different methods for geocoding residence and found that geocoding was able to locate 72.9% of individuals to within 250 meters of their true compound location. These findings provide valuable insights into the effectiveness of geolocation strategies for disease control in rural areas.
# """

# evidence_sentence_1 = f"""
# Geolocation strategies were tested at rural health facilities in western Kenya.
# Geocoding was able to locate 72\u00b79% [95% confidence interval (CI) 67\u00b77\u201377\u00b76] of individuals to within 250 m of the true compound location.
# These strategies tested provide options for quickly obtaining spatial information on individuals presenting at health facilities.
# """

# A newly discovered virus in Russian bats, called Khosta-2, shares similarities with SARS-CoV-2 and has the potential to infect humans. The spike proteins of Khosta-2 can infect human cells and are resistant to current vaccines. This finding emphasizes the need for universal vaccines that can protect against all sarbecoviruses, rather than just specific variants of SARS-CoV-2. While most sarbecoviruses cannot infect human cells, Khosta-2 poses a potential threat. The virus can attach to a receptor protein found in human cells using its spike protein, and current vaccines do not neutralize it. Although Khosta-2 lacks certain genes involved in human pathogenesis, there is a risk of it combining with other viruses to create a potentially more dangerous virus.
# OBJECTIVES: This study aimed to examine how Americans’ opinions of the seriousness of various health-related problems have changed over time and to quantify the public’s preferences for research prioritization. METHODS: We conducted a survey that asked respondents to rate the seriousness of 80 health-related problems on a 4-point Likert scale (“very serious problem,” “somewhat serious problem,” “not too serious of a problem,” or “not a problem at all”). Results were compared with past surveys from 2001 and 2013 that examined the same set of health-related problems (with the exception of COVID-19). The survey also included best-worst scaling questions that asked respondents to select, from 20 health problems, those they considered most and least important for research funding. Respondents were recruited from the KnowledgePanel, a nationally representative sample of American households. RESULTS: A total of 768 adults completed the survey between September 3, 2020, and September 14, 2020. The health-related problems that Americans consider to be “very serious” generally align with the leading causes of death and noncommunicable diseases such as heart disease, diabetes, and mental health; nevertheless, several social determinants of health are also identified. COVID-19 was an unsurprising top priority, whereas cancer remains the highest and a persistent priority for research funding. CONCLUSIONS: Americans consider a diverse set of health-related problems to be “very serious,” with recognition of social determinants of health rising. Our findings offer guidance as to the disease areas for which the public would value further public and private investment in treatment innovations.COVID 19, caused by the SARS-CoV-2 virus, a newly discovered coronavirus, has caused the global pandemic of early 2020 The first case was described in December 2019 in Wuhan, China, and by March 2020, most countries around the world have put in place some of the strictest restrictions seen in decades in order to slow down the spread of the disease Patients with preexisting hypertension and cardiovascular comorbidities were reported to be at an increased risk of serious infections caused by SARS-CoV-2 Considering that those are among the most common chronic medical conditions in the Western world, the potential impact of it is huge The proposed mechanism behind those associations is the expression of angiotensin converting enzyme II (ACE II) in those patients Furthermore, the association between ACE inhibitors/AR blockers, which are among the most frequently prescribed medications, and serious cases of COVID 19 has been studied with the same mechanism in mind The reports on the association between hypertension and COVID 19 morbidity and mortality are less clear, and the International Society of Hypertension even claims that there is none The reports on the association between heart failure or coronary disease and COVID 19 are more uniform, and all seem to point to a greater risk from serious infections faced by patients with those comorbidities A significant effort will need to be invested by the scientific community into finding strategies for protecting those patients from contracting the virus in the first place and then, once infected, into developing management plans aimed at preserving cardiac function as much as possibleCOVID-19 has caused immense social and economic losses throughout the world. Subjects recovered from COVID are learned to have complications. Some studies have shown a change in the heart rate variability (HRV) in COVID-recovered subjects compared to the healthy ones. This change indicates an increased risk of heart problems among the survivors of moderate-to-severe COVID. Hence, this study is aimed at finding HRV features that get altered in COVID-recovered subjects compared to healthy subjects. Data of COVID-recovered and healthy subjects were collected from two hospitals in Delhi, India. Seven ML models have been built to classify healthy versus COVID-recovered subjects. The best-performing model was further analyzed to explore the ranking of altered heart features in COVID-recovered subjects via AI interpretability. Ranking of these features can indicate cardiovascular health status to doctors, who can provide support to the COVID-recovered subjects for timely safeguard from heart disorders. To the best of our knowledge, this is the first study with an in-depth analysis of the heart status of COVID-recovered subjects via ECG analysis. © 2021 IEEE.

In [8]:
news_paragraph_1 = f"""
Exercise therapy has been found to be ineffective in improving ambulatory ability in a new model of peripheral arterial disease (PAD), according to a groundbreaking study published in the New England Journal of Medicine. The study used a two-stage procedure in mice, resulting in more severe and sustained ischemia than current models. The mice were unable to walk again after the exercise session, challenging the conventional wisdom on the treatment of PAD. The findings highlight the need for new approaches and therapies for PAD patients.
"""
evidence_abstract_1 = f"""
Nigeria has the highest number of AIDS-related deaths in the world. In this study, we characterised the HIV-1 molecular epidemiology by analysing 1442 HIV-1 pol sequences collected 1999–2014 from four geopolitical zones in Nigeria using state-of-the-art maximum-likelihood and Bayesian phylogenetic analyses. The main circulating forms were the circulating recombinant form (CRF) 02_AG (44% of the analysed sequences), CRF43_02G (16%), and subtype G (8%). Twenty-three percent of the sequences represented unique recombinant forms (URFs), whereof 37 (11%) could be grouped into seven potentially novel CRFs. Bayesian phylodynamic analysis suggested that five major Nigerian HIV-1 sub-epidemics were introduced in the 1960s and 1970s, close to the Nigerian Civil War. The analysis also indicated that the number of effective infections decreased in Nigeria after the introduction of free antiretroviral treatment in 2006. Finally, Bayesian phylogeographic analysis suggested gravity-like dynamics in which virus lineages first emerge and expand within large urban centers such as Abuja and Lagos, before migrating towards smaller rural areas. This study provides novel insight into the Nigerian HIV-1 epidemic and may have implications for future HIV-1 prevention strategies in Nigeria and other severely affected countries.The present study aims to investigate the genomic variability and epidemiology of SARS-CoV-2 in Pakistan along with its role in the spread and severity of infection during the three waves of COVID-19. A total of 453 genomic sequences of Pakistani SARS-CoV-2 were retrieved from GISAID and subjected to MAFFT-based alignment and QC check which resulted in removal of 53 samples. The remaining 400 samples were subjected to Pangolin-based genomic lineage identification. And to infer our SARS-CoV-2 time-scaled and divergence phylogenetic trees, 3804 selected global reference sequences plus 400 Pakistani samples were used for the Nextstrain analysis with Wuhan/Hu-1/2019, as reference genome. Finally, maximum likelihood based phylogenetic tree was built by using the Nextstrain and coverage map was created by employing Nextclade. By using the amino acid substitutions, the maximum likelihood phylogenetic trees were developed for each wave, separately. Our results reveal the circulation of 29 lineages, belonging to following seven clades G, GH, GR, GRY, L, O, and S in the three waves. From first wave, 16 genomic lineages of SARS-CoV-2 were identified with B.1(24.7%), B.1.36(18.8%), and B.1.471(18.8%) as the most prevalent lineages respectively. The second wave data showed 18 lineages, 10 of which were overlapping with the first wave suggesting that those variants could not be contained during the first wave. In this wave, a new lineage, AE.4, was reported from Pakistan for the very first time in the world. However, B.1.36 (17.8%), B.1.36.31 (11.9%), B.1.1.7 (8.5%), and B.1.1.1 (5.9%) were the major lineages in second wave. Third wave data showed the presence of nine lineages with Alpha/B.1.1.7 (72.7%), Beta/B.1.351 (12.99%), and Delta/B.1.617.2 (10.39%) as the most predominant variants. It is suggested that these VOCs should be contained at the earliest in order to prevent any devastating outbreak of SARS-CoV-2 in the country.The present study aims to investigate the genomic variability and epidemiology of SARS-CoV-2 in Pakistan along with their role in the spread and severity of infection during the three waves of COVID-19. A total of 453 genomic sequences of Pakistani SARS-CoV-2 were retrieved from GISAID and subjected to MAFFT-based alignment and QC check which resulted in removal of 53 samples. The remaining 400 samples were subjected to Pangolin-based genomic lineage identification. And to infer our SARS-CoV-2 time-scaled and divergence phylogenetic trees, 3,804 selected global reference sequences plus 400 Pakistani samples were used for the Nextstrain analysis with Wuhan/Hu-1/2019, as reference genome. Finally, maximum likelihood based phylogenetic tree was built by using the Nextstrain & coverage map was created by employing Nextclade. And by using the amino acid subsitutions the maximum likelihood phylogenetic trees were developed for each wave, separately. Our results reveal the circulation of 29 lineages, belonging to following 7 clades G, GH, GR, GRY, L, O, & S in the three waves. From first wave, 16 genomic lineages of SARS-CoV-2 were identified with B.1(24.7%), B.1.36(18.8%), & B.1.471(18.8%) as the most prevalent lineages respectively. The second wave data showed 18 lineages, 10 of which were overlapping with the first wave suggesting that those variants could not be contained during the first wave. In this wave, a new lineage, AE.4, was reported from Pakistan for the very first time in the world. However, B.1.36 (17.8%), B.1.36.31 (11.9%), B.1.1.7 (8.5%) & B.1.1.1 (5.9%) were the major lineages in second wave. Third wave data showed the presence of 9 lineages with Alpha/B.1.1.7 (72.7%), Beta/B.1.351 (12.99%), & Delta/B.1.617.2 (10.39%) as the most predominant variants. It is suggested that these VOCs should be contained at the earliest in order to prevent any devastating outbreak of SARS-CoV-2 in the country.
"""

In [9]:
news_paragraph_2 = f"""
A groundbreaking study has revealed the key role of the renin-angiotensin system (RAS) in lung cancer. The study found that the RAS plays a pleiotropic role in the lung, particularly through locally regulated alternative molecules and secondary pathways. Angiotensin peptides are involved in critical biological processes in the lung tumor microenvironment, such as cell proliferation, immunoinflammatory response, hypoxia, and angiogenesis. These findings suggest that blocking the RAS pathway may serve as an adjuvant therapy in lung cancer.
"""
evidence_abstract_2 = f"""
The mechanistic involvement of the renin-angiotensin system (RAS) reaches beyond cardiovascular physiopathology. Recent knowledge pinpoints a pleiotropic role for this system, particularly in the lung, and mainly through locally regulated alternative molecules and secondary pathways. Angiotensin peptides play a role in cell proliferation, immunoinflammatory response, hypoxia and angiogenesis, which are critical biological processes in lung cancer. This manuscript reviews the literature supporting a role for the renin-angiotensin system in the lung tumor microenvironment and discusses whether blockade of this pathway in clinical settings may serve as an adjuvant therapy in lung cancer.SIMPLE SUMMARY: Glioblastoma (GB) is the most aggressive brain cancer in humans. Patient survival outcomes have remained dismal despite intensive research over the past 50 years, with a median overall survival of only 14.6 months. We highlight the critical role of the renin–angiotensin system (RAS) on GB cancer stem cells and the tumor microenvironment which, in turn, influences cancer stem cells in driving tumorigenesis and treatment resistance. We present recent developments and underscore the need for further research into the GB tumor microenvironment. We discuss the novel therapeutic targeting of the RAS using existing commonly available medications and utilizing model systems to further this critical investigation. ABSTRACT: Glioblastoma (GB) is an aggressive primary brain tumor. Despite intensive research over the past 50 years, little advance has been made to improve the poor outcome, with an overall median survival of 14.6 months following standard treatment. Local recurrence is inevitable due to the quiescent cancer stem cells (CSCs) in GB that co-express stemness-associated markers and components of the renin–angiotensin system (RAS). The dynamic and heterogeneous tumor microenvironment (TME) plays a fundamental role in tumor development, progression, invasiveness, and therapy resistance. There is increasing evidence showing the critical role of the RAS in the TME influencing CSCs via its upstream and downstream pathways. Drugs that alter the hallmarks of cancer by modulating the RAS present a potential new therapeutic alternative or adjunct to conventional treatment of GB. Cerebral and GB organoids may offer a cost-effective method for evaluating the efficacy of RAS-modulating drugs on GB. We review the nexus between the GB TME, CSC niche, and the RAS, and propose re-purposed RAS-modulating drugs as a potential therapeutic alternative or adjunct to current standard therapy for GB.BACKGROUND The Renin-Angiotensin System (RAS) comprises a complex molecular cascade with two counter-regulatory axes, the classical and the alternative. Angiotensin II and Angiotensin-(1-7), the main peptides of the RAS, exert opposite effects in multiple organs and systems, including the cardiovascular, renal, pulmonary and immune systems. Strong evidence supports the hypothesis of a local RAS in the Central Nervous System (CNS) and its modulatory roles in neuroendocrinology and neurotransmission. OBJECTIVE In this narrative review, we provide a comprehensive approach on experimental and clinical data regarding RAS molecules expression and possible roles in physiology and physiopathology of CNS diseases. METHODS This non-systematic review summarizes evidence on RAS implications in CNS diseases and its possible relationships with COVID-19. RESULTS We divided the possible RAS mechanisms in distinct conditions during lifespan, approaching from congenital infections to neurodegenerative alterations, passing through mood disorders and cerebrovascular diseases. We also gathered current evidence about the possible effects of RAS in Covid-19, particularly the described cases that course with neurological manifestations. CONCLUSION Although there are limitations and controversies, the analysis of RAS mechanisms in the CNS certainly represents an interesting field of research. However, further investigation is necessary to support the noteworthy interactions and provide a better comprehension of the cross-talk between RAS and the CNS. Investigations in this research field may shed lights to novel therapeutic targets.
"""

In [10]:
example_1 = f""" 
Example 1: 
    News Paragraph: {news_paragraph_1}
    Evidence: {evidence_abstract_1}
    Expected Output:
    {{
        "prediction": "False",
        "reason": "The evidence abstracts provided do not directly address the central claim of the news paragraph, which is about the ineffectiveness of exercise therapy in improving ambulatory ability in peripheral arterial disease (PAD). The evidence abstracts discuss HIV-1 molecular epidemiology in Nigeria and the genomic variability and epidemiology of SARS-CoV-2 in Pakistan. Therefore, there is no evidence to support or refute the central claim of the news paragraph."
    }} 
"""

example_2 = f""" 
Example 2: 
    News Paragraph: {news_paragraph_2}
    Evidence: {evidence_abstract_2}
    Expected Output:
    {{
        "prediction": "True",
        "reason": "The evidence abstracts support the key role of the renin-angiotensin system (RAS) in lung cancer, as mentioned in the news paragraph. The abstracts discuss the pleiotropic role of RAS in the lung, its involvement in critical biological processes in the lung tumor microenvironment, and the potential of blocking the RAS pathway as an adjuvant therapy in lung cancer."
    }} 
"""

In [11]:
#     Determine if the evidence supports or refutes the point of view in the news paragraph.\
#     Finally provide an answer of 'True' if the news paragraph is factually true or \
#     'False' if it is false based on the evidence with reasons. \
#     Provide the final output in a JSON format with the keys: prediction and reason.

In [12]:
i = 0
generated_result = []

In [13]:
for item in data:
    print(f"Predicting {i}")
    
    news = item['abstractive']
    #print(news)
    
    evidence_list = []
    if type(item['evidence'])==str:
        evidence_list.append(item['evidence'])
    else:
        evidence_list.extend(item['evidence'])
    print(f"evidence list: {evidence_list}")
    
    evidence = [it['abstract'] for it in evidence_data if it['cord_uid'] in evidence_list]
    if evidence:
        evidence = evidence[0]
    else:
        evidence = ""
#     evidence = ''.join(evidence)
    
#     user_message = f""" Analyze the following news and evidences to determine their relationship. Example 1: {example_1} and Example 2: {example_2}
#     Here is a news behind {delimiter_1}{news} and evidence behind {delimiter_2}{evidence}.
#     Decide if the evidence support or refute the point of view of the news paragraph and provide an \
#     answer of 'True' if the news is factually true or 'False' if it is false with reasons.\
#     Provide the response in JSON format with the keys - 'prediction' and 'reason'.\ """
    
#     assistant_message = """ As a Fact Checker, analyze the evidence and the given examples and provide the result in JSON format.\
#     Refer sentences from news and evidence to support your answer. """

    user_message = f"""
    Task: Analyze the following news paragraphs and the relevant evidences to determine their relationship.

    Example 1: {example_1}
    Example 2: {example_2}

    Now analyze the following:
    News Paragraph: {delimiter_1}{news}
    Relevant Evidences: {delimiter_2}{evidence}
    
    Instructions: Decide if the relevant evidence support or refute the point of view of the news paragraph.\
    Finally provide an answer of 'True' if the news paragraph is factually true or 'False' if it is false based on the evidence.\
    Then, explain your reasoning in a few sentences. Output: Format your response as JSON with two keys - 'prediction' and 'reason'. \
    The 'prediction' key should have the value 'True' or 'False', and the 'reason' key should contain the explanation for your choice.\
    Example of Expected Output:
    {{
        "prediction": "True",
        "reason": "The relevant sentences provide evidence that corroborates the main point made in the news paragraph by presenting similar findings..."
    }}
    """
    
    assistant_message = """
    As a Fact Checker, carefully analyze the evidence abstracts and the given examples and provide the result in JSON format. \
    Reference specific phrases or sentences from both the news story and the evidence to support your prediction.
    """
        
    messages =  [ {'role':'system', 'content': system_message},    
                  {'role':'user', 'content': user_message},  
                  {'role': 'assistant', 'content': assistant_message}]    
    
    response = get_completion_from_messages(messages)
    print(response)
    
    result = dict()
    result['news'] = news
    result['evidence list'] = evidence_list
    result['ground truth'] = item['label']
    
    try:
        result.update(json.loads(response))
        
    except JSONDecodeError as e:
        pattern = r'"prediction": "(.*?)",\s*"reason": "(.*?)"'
        match = re.search(pattern, response)
        if match:
            result['prediction'] = match.group(1)
            result['reason'] = match.group(2)
#             print(dict({'prediction':result['prediction'], 'reason':result['reason']})) 
    
    generated_result.append(result)
    
    time.sleep(2)
    i += 1
    
#     break

Predicting 0
evidence list: ['01tncjq0', '1z1912bm', 'q5qc70jb']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the central claim of the news paragraph. The evidence abstracts discuss the testing of geolocation strategies in rural areas of western Kenya, which were able to accurately locate individuals to within a certain distance of their true compound location. These findings align with the claim that geolocation strategies can accurately track disease distribution in rural areas where formal address networks do not exist."
}
Predicting 1
evidence list: ['02wvru99', 'n8teg5sq', '19sejitq']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstract states that the study compared the efficacy of oseltamivir-peramivir combination therapy versus oseltamivir monotherapy in H7N9 virus infection and found no significant difference in the duration of treatment or viral load decline between t

Predicting 12
evidence list: ['08ds967z', 'row2mn17', 'r6k3uk69']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the news paragraph's claim that scientists have made a breakthrough discovery in the Middle East, identifying a new coronavirus known as Middle East respiratory syndrome human coronavirus (MERS-CoV). The evidence abstract mentions the identification of MERS-CoV in patients with severe acute respiratory infection and subsequent renal failure resulting in death. It also confirms the number of laboratory-confirmed cases and deaths associated with MERS-CoV. Therefore, the evidence aligns with the central claim of the news paragraph."
}
Predicting 13
evidence list: ['08fkra10', '08fkra10', 'gsxzk54i']
{
    "prediction": "True",
    "reason": "The evidence abstracts support the key points mentioned in the news paragraph. The abstracts discuss the presentations at the 8th International Conference on Management and Rehabilitation of Chronic Respiratory Fa

Predicting 23
evidence list: ['0o6agnrc', 'm0hpbw6i', 'yaahrjs4']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstract states that the study analyzed the structure and function of the key proteins of the 2019-nCoV and found no significant changes in important proteins involved in the virus's entry into host cells. This aligns with the claim made in the news paragraph that the key proteins of the virus remain unchanged. Therefore, the news paragraph is factually true based on the evidence provided."
}
Predicting 24
evidence list: ['0oa4x05s', '1e1b3zre', '2idsg0hi']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the news paragraph's claim that cowpox virus outbreaks occurred in cheetahs at a safari park in Denmark. The evidence abstract confirms the occurrence of five cowpox virus outbreaks in cheetahs at the same safari park between 2010 and 2014. It also mentions that nine cheet

Predicting 34
evidence list: ['10bu7iwg', '10bu7iwg', 'yc2y3d7m']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstracts discuss the effectiveness of interferon gamma as a treatment or prevention option for Ebola virus outbreaks. They mention that administering interferon gamma before or after infection protected mice and reduced the severity of the virus. The evidence also states that the drug inhibited Ebola virus infection of macrophages, which is a key target of the virus. These findings align with the claim made in the news paragraph that interferon gamma could be an effective treatment or prevention option for Ebola virus outbreaks."
}
Predicting 35
evidence list: ['10j9cp9c', 'lw3yoayu', 'pfcze9iu']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the central claim of the news paragraph. The evidence abstract discusses the development of a rapid and cost-effective method for 

Predicting 46
evidence list: ['1do4bqtf', 'o59zy2p6', '2cppzng4']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstract states that the occurrence of multi-drug resistant bacteria remains rare among healthy dogs in Sweden, with only 0.9% of the dogs tested carrying multi-resistant ESBL-producing Escherichia coli. This aligns with the claim made in the news paragraph that multi-drug resistant bacteria is rare among healthy dogs in the country."
}
Predicting 47
evidence list: ['1ey6ie95', '826dikmh', 'ybdj5hpi']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the main point made in the news paragraph. The evidence abstract discusses the study's findings that AKAP7 is significantly associated with post-stroke brain barrier disruption, particularly 24 hours after hospital admission. The evidence also mentions the correlation between AKAP7 and ITGA3, and the experiments conducted in vit

...

Predicting 1146
evidence list: ['o4k30ki3', 'vqltdvrx', 'ijqwc7dz']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the key points made in the news paragraph. The evidence abstract discusses the prioritization of population groups for COVID-19 vaccination in China, which aligns with the central claim of the news paragraph. The evidence suggests prioritizing essential workers in the early phase of the vaccination program, followed by older adults, individuals with underlying health conditions, pregnant women, adults without underlying health conditions, and children in subsequent phases. The evidence also mentions the estimated time it would take to vaccinate 70% of the overall population in China. Therefore, the evidence supports the factual accuracy of the news paragraph."
}
Predicting 1147
evidence list: ['8fgwrevs', '8fgwrevs', 'oltfopmh']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the central claim of the news paragraph. The 

Predicting 1157
evidence list: ['svjem6g1', '27mfi20z', '27mfi20z']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstract states that in individuals with a previous SARS-CoV-2 infection, a single dose of mRNA vaccine is immunologically equivalent to a full vaccine schedule in naive individuals. This aligns with the news paragraph's claim that a single dose of the mRNA vaccine is sufficient for achieving immunity in all individuals, regardless of prior infection. Additionally, the evidence mentions that antibody levels after a single vaccine injection were significantly higher in individuals with previous infection compared to those who received a full vaccination schedule without prior infection, further supporting the claim. The evidence also mentions that adverse events were more frequent after the second dose of the vaccine, which aligns with the news paragraph's statement. Therefore, based on the eviden

Predicting 1168
evidence list: ['jztjnh0v', 'rjyrazm2', '4g5xd2ys']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the central claim of the news paragraph. The evidence abstract discusses the duration of clearance of SARS-CoV-2 in respiratory samples and the reduction of viral load in samples with high or intermediate viral loads. This aligns with the news paragraph's claim that the study successfully converted cycle threshold (Ct) values into copies/μL of the virus, allowing for more accurate assessment of viral load. Additionally, the evidence abstract mentions the potential seasonal pattern of viral loads, which is consistent with the news paragraph's finding of significantly lower viral loads during the summer inter-epidemic waves in Italy. The evidence also supports the claim of the study establishing a predictive model of case-fatality probability based on viral load, gender, and age. Therefore, the news paragraph is factually true based on the evidence

Predicting 1180
evidence list: ['d2qrz98c', 'xg3gfpb9']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstract discusses the struggles associated with vaccination, such as adverse reactions, vaccine shortage in poor regions, and reduced effectiveness against new variants. It also suggests the mix-and-match strategy as a solution to these struggles. This aligns with the news paragraph's claim that plant-made vaccines offer a scalable solution to meet the global demand for vaccines and can rapidly respond to emerging viruses. Therefore, the news paragraph is factually true based on the evidence provided."
}
Predicting 1181
evidence list: ['6dvbohvq', '34mk0ejd', 'mo90z8hz']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the news paragraph's claim that there is a connection between COVID-19 and coagulation dysfunction in adults. The evidence abstract discusses a meta-analysis of 13 st

Predicting 1192
evidence list: ['j2tm0ji5', 'xizlc7rf', '5c91xnhz']
{
    "prediction": "False",
    "reason": "The evidence abstracts provided do not directly address the central claim of the news paragraph, which is about the intentional release of the COVID-19 virus as part of a covert operation. The evidence abstracts discuss a pneumonia outbreak caused by the SARS-CoV-2 virus and describe the clinical manifestations and treatment. Therefore, there is no evidence to support or refute the central claim of the news paragraph."
}
Predicting 1193
evidence list: ['gfskwmh4', 'oe3puu01', 'h0mgiexr']
{
    "prediction": "True",
    "reason": "The relevant evidence supports the claim made in the news paragraph. The evidence abstract discusses the availability, quality, and inclusivity of clinical guidelines produced during the early stage of the COVID-19 pandemic. It mentions that the guidelines were of remarkable quality and inclusivity, providing clear guidance on the use of antiviral dr

In [14]:
i

1200

In [15]:
i

1200

In [16]:
len(generated_result)

1200

In [28]:
# generated_result.remove(generated_result[-1])

In [21]:
generated_result
# for item in data:
# #     print(item['abstractive'])
#     print(item)
#     break

[{'news': 'A recent study conducted at St. George\'s Hospital in London found that mask mandates had "no discernible difference" in COVID-19 transmission rates in hospitals. The data also showed that infection rates did not increase when mask mandates were not in place in National Health Service (NHS) facilities during the surge of omicron cases. However, health experts argue that this does not mean face masks are "worthless" and emphasize the need for "rational and proportionate" masking policies in hospitals for future coronavirus flare-ups. The study analyzed infection control data over a 40-week period and found that lifting mask mandates did not significantly change the hospital-acquired COVID-19 infection rate. Despite the evidence, face masks have become a symbol of the COVID-19 culture wars, and skepticism remains due to mixed messages and inconclusive studies on their effectiveness. The study\'s authors suggest using their findings to inform mask policies in healthcare setting

In [76]:
# data[537]['prediction'] = "refute"
# data[537]['reason'] = "The relevant sentences refute the point of view presented in the news paragraph. The news paragraph claims that there is evidence that cancers are occurring in ways never before seen after people receive COVID-19 vaccinations, and that the Covid-19 vaccine has compromised immune systems by disabling the body's ability to fight non-normal cancerous cells. However, none of the relevant sentences mention anything about cancers occurring in new ways or the Covid-19 vaccine compromising immune systems. Instead, the relevant sentences discuss formalisation in Isabelle/HOL, wider array of vascular phenomena related to Covid vaccinations, and the belief that a healthy immune system can fight and disable cancerous cells. These sentences do not provide any supporting evidence for the claims made in the news paragraph, and therefore refute its point of view."

In [17]:
with open('final_output_GPT35_llm_fewshot.json', 'w') as file:
    json.dump(generated_result, file)

In [26]:
# data[515]